In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch
import torchvision
from torchvision import models
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import os

import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, FastICA
from sklearn.covariance import LedoitWolf, MinCovDet

from torchvision.transforms import transforms
from sklearn.metrics import roc_auc_score

np.random.seed(252525)
torch.manual_seed(252525)

import torch
import torch.nn as nn

from data.mvtec import *
from hugeica import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
sys.path.append("../../gaussian-ad-mvtec/src/gaussian/")
sys.path.append("../../gaussian-ad-mvtec/src/common/utils")

from transparent import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
bs = 50
clazz = 3
epochs = 1
augment = False
layer = 5

net, input_size = build_transparent(
            "efficientnet-b4",
            pretrained=True,
            extract_blocks=[0,1,2,3,4,5,6,7],
            freeze=True,
        )

net = net.to(device)

X_, X_valid_, X_test_, X_labels_, T = zip(*[dataloader(clazz, P=224, s=224, label_per_patch=False, augment=augment) for i in range(epochs)])
X_, X_valid_, X_test_ = np.concatenate(X_), np.concatenate(X_valid_), np.concatenate(X_test_)

    
X_ = torch.cat([ net(torch.from_numpy( X_[i:i+bs] ).to(device) ).detach().cpu()[layer] for i in range(0, len(X_), bs)]).cpu().numpy()
X_valid_ = torch.cat([ net(torch.from_numpy( X_valid_[i:i+bs] ).to(device) ).detach().cpu()[layer] for i in range(0, len(X_valid_), bs)]).cpu().numpy()
X_test_ = torch.cat([ net(torch.from_numpy( X_test_[i:i+bs] ).to(device) ).detach().cpu()[layer] for i in range(0, len(X_test_), bs)]).cpu().numpy()

X_.mean(),X_.std()

#mu = X_.mean(0)
c = X_.shape[1]
n = X_.shape[0]
X__ = X_.transpose(0,2,3,1) # B, H, W, C
#X__ = X__.reshape(n, -1, c).mean(1)
print(X__.shape)
X__ = X__.reshape(len(X__), -1)
print(X__.shape)

#pca = PCA(n_components=0.95, whiten=False).fit(X__)
#class pca:
#    transform = lambda x : x
#X__ = pca.transform(X__)

C_inv = LedoitWolf().fit(X__).precision_
mu = X__.mean(0)

# Pooling inliers
n_valid = len(X_valid_)
#X_valid_ = X_valid_.mean((2,3))
X_valid_ = X_valid_.reshape(n_valid, -1)

# Pooling outliers
n_test = len(X_test_)
X_test_ = X_test_.reshape(n_test, -1)
#X_test_ = X_test_.mean((2,3))

# Mahalanobis
X_valid_ = X_valid_ - mu
X_test_ = X_test_ - mu
M_valid_ = ( X_valid_ * ( C_inv @ X_valid_.T ).T ).sum(1)    
M_test_  = ( X_test_ * ( C_inv @ X_test_.T ).T ).sum(1)    
 

scores_inliers = []
scores_outliers = []

scores_inliers.append(M_valid_)
scores_outliers.append(M_test_)

auc = roc_auc_score([0] * n_valid + [1] * n_test, np.concatenate([np.asarray(scores_inliers).sum(0), np.asarray(scores_outliers).sum(0)]))
print(MVTEC.CLASSES[clazz], auc)

Loaded pretrained weights for efficientnet-b4
(267, 14, 14, 160)
(267, 31360)


In [2]:
class Add(nn.Module):
    
    def __init__(self, dim):
        super().__init__()
        self.register_parameter(name='mean', param=nn.Parameter(torch.zeros(dim).to(device)))
        
    def forward(self, X):
        return X + self.mean
    
    
class Matmul(nn.Module):
    
    def __init__(self, dim):
        super().__init__()
        self.register_parameter(name='sphering', param=nn.Parameter(torch.eye(dim).to(device)))
        
    def forward(self, X):
        return torch.matmul(X, self.sphering)

class FeatureExtractor(nn.Module):

    def __init__(self, pooling=4):
        super().__init__()
        #
        # Preparation
        #
        # Let's try to find the pooling layer positions
        net = torch.hub.load('pytorch/vision:v0.10.0', 'vgg19', pretrained=True).features
        net = net.to(device)
        net.eval()
        
        self.idx_pooling_layer = []
        for i, module in enumerate( net.modules() ):
            if not isinstance(module, nn.Sequential):
                if isinstance(module, nn.MaxPool2d):
                    self.idx_pooling_layer.append(i)
        self.F = self.idx_pooling_layer[pooling]
        
        #
        # The final net
        #
        self.features = net[:self.F]
        self.shape = self.features(torch.normal(0,1, (1, 3, 224, 224)).to(device)).shape
        #self.mean = Add(self.shape[1])       
        #self.sphering = Matmul(self.shape[1])
        
        self.mean = nn.Linear(self.shape[1], self.shape[1], bias=True).to(device)
        self.sphering = nn.Linear(self.shape[1], self.shape[1], bias=False).to(device)
        
        self.reset()
        
        # Sometimes inplace ReLUs cause problems with Laplace
        for module in self.features.modules():
            if not isinstance(module, nn.Sequential):
                if isinstance(module, nn.ReLU):
                    module.inplace = False

    def reset(self):
        self.sphering.weight.data = torch.eye(self.shape[1]).to(device)
        self.mean.weight.data = torch.eye(self.shape[1]).to(device)
        self.mean.bias.data = torch.zeros(self.shape[1]).to(device)
    
    def forward(self, X):
        X = self.features(X)
        X = X.mean((2,3)) # pooling the latents
        X = self.mean(X)
        X = self.sphering(X)
        return X

net = model = FeatureExtractor()
net

Using cache found in /home/matthias/.cache/torch/hub/pytorch_vision_v0.10.0


FeatureExtractor(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU()
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU()
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU()
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU()
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU()
    (18): MaxPool2d(kernel_size=2, stride=

In [3]:
from laplace import Laplace
from laplace.utils import ModuleNameSubnetMask

In [ ]:
bs = 25
clazz = 13
epochs = 50
augment = True
layer = 6

net = FeatureExtractor(pooling=0)
net = net.to(device)
net.eval()

for clazz in range(15):

    net.reset()

    #####################################################
    #
    # LOAD THE DATA
    #####################################################
    X_, X_valid_, X_test_, X_labels_, T = zip(*[dataloader(clazz, P=224, s=224, label_per_patch=False, augment=augment) for i in range(epochs)])
    X_, X_valid_, X_test_ = np.concatenate(X_), np.concatenate(X_valid_), np.concatenate(X_test_)

    with torch.no_grad():
        X__ = torch.cat([ net(torch.from_numpy( X_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_), bs)]).cpu().numpy()
        X_valid__ = torch.cat([ net(torch.from_numpy( X_valid_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_valid_), bs)]).cpu().numpy()
        X_test__ = torch.cat([ net(torch.from_numpy( X_test_[i:i+bs] ).to(device) ).detach().cpu() for i in range(0, len(X_test_), bs)]).cpu().numpy()

    #####################################################
    #
    # SPHERING FEATURE SPACE
    #####################################################
    k = 20
    k_max = np.min(X__.shape)
    pca = PCA(n_components=k_max, whiten=True).fit(X__)
    sphering = pca.components_[-k:] / np.sqrt( pca.explained_variance_ )[-k:, None]
    #sphering = pca.components_[:k] / np.sqrt( pca.explained_variance_ )[:k, None]
    mu = X__.mean(0)
    X__ = (sphering @ (X__ - mu).T).T

    #sphering = pca.components_ / np.std(pca.explained_variance_)
    net.mean.bias.data = -torch.from_numpy(mu).to(device)
    # >>>>>>>>>> net.mean.weight.data = torch.eye(k).to(device)
    net.sphering.weight.data = torch.from_numpy(sphering).contiguous().to(device)

    #####################################################
    #
    # LAPLACE APPROXIMATION
    #####################################################
    lap_bs = 5
    # >>>>>>>>>> k = len(mu)
    X__ = torch.from_numpy(X_)
    y__ = torch.zeros((len(X_), k))
    dataset = torch.utils.data.TensorDataset(X__, y__)
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=lap_bs)

    model = net
    subnetwork_mask = ModuleNameSubnetMask(model, module_names=["features.0"])
    subnetwork_mask.select()
    subnetwork_indices = subnetwork_mask.indices.to("cpu")

    la = Laplace(model, subset_of_weights='last_layer', # diag', 'kron', 'full', 'lowrank' {'last_layer', 'subnetwork', 'all'}
                 hessian_structure='full', likelihood='regression',
                 subnetwork_indices=subnetwork_indices
                )
    la.fit(train_loader)

    #####################################################
    #
    # COMPUTE SCORES
    #####################################################
    f_mu_in, f_var_in   = zip(*[la(torch.from_numpy( X_valid_[i:i+lap_bs] ).to(device) ) for i in range(0, len(X_valid_), lap_bs)])
    f_mu_in, f_var_in   = torch.cat(f_mu_in).cpu().numpy(), torch.cat(f_var_in).cpu().numpy()
    
    f_mu_out, f_var_out = zip(*[la(torch.from_numpy( X_test_[i:i+lap_bs] ).to(device) ) for i in range(0, len(X_test_), lap_bs)])
    f_mu_out, f_var_out = torch.cat(f_mu_out).cpu().numpy(), torch.cat(f_var_out).cpu().numpy()

    # Aggregate scores
    scores_inliers = []
    scores_outliers = []
    scores_inliers.append(np.linalg.norm(f_mu_in, axis=1))
    scores_outliers.append(np.linalg.norm(f_mu_out, axis=1))

    auc = roc_auc_score([0] * len(f_mu_in) + [1] * len(f_mu_out), np.concatenate([np.asarray(scores_inliers).sum(0), np.asarray(scores_outliers).sum(0)]))
    print("PPD-mean (AUC)", MVTEC.CLASSES[clazz], auc)

    # Aggregate scores
    scores_inliers = []
    scores_outliers = []
    scores_inliers.append(f_var_in.sum((1,2)))
    scores_outliers.append(f_var_out.sum((1,2)))

    auc = roc_auc_score([0] * len(f_mu_in) + [1] * len(f_mu_out), np.concatenate([np.asarray(scores_inliers).sum(0), np.asarray(scores_outliers).sum(0)]))
    print("PPD-var (AUC)", MVTEC.CLASSES[clazz], auc)

Using cache found in /home/matthias/.cache/torch/hub/pytorch_vision_v0.10.0


NameError: name 'auc' is not defined